In [9]:
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
"""
# example to show working of stopwords and tokenization library in python
file_content_temp = "my Name is Ayush fir.st . second. .ninth"  #note: period after a word is removed but period before a word is not removed
tokens_temp = nltk.word_tokenize(file_content_temp)
stop_words_temp = stopwords.words('english')
punctuation_list_temp=['\'','.',')','(','{','}','[',']','\,','?','!','\'\'',':',';','...']
tokens_temp = [word.lower() for word in tokens_temp if (word not in stop_words_temp and word not in punctuation_list_temp and word.isnumeric()==False)]
RESULT=['name', 'ayush', 'fir.st', 'second', '.ninth']

#EXAMPLE TO SHOW WORKING OF ASCII REMOVAL IN PYTHON
string_with_nonASCII = "àa string withé fuünny charactersß."
encoded_string = string_with_nonASCII.encode("ascii", "ignore")
decode_string = encoded_string.decode()
print(decode_string)
RESULT="a string with funny characters."

"""

'\n# example to show working of stopwords and tokenization library in python\nfile_content_temp = "my Name is Ayush fir.st . second. .ninth"  #note: period after a word is removed but period before a word is not removed\ntokens_temp = nltk.word_tokenize(file_content_temp)\nstop_words_temp = stopwords.words(\'english\')\npunctuation_list_temp=[\'\'\',\'.\',\')\',\'(\',\'{\',\'}\',\'[\',\']\',\'\\,\',\'?\',\'!\',\'\'\'\',\':\',\';\',\'...\']\ntokens_temp = [word.lower() for word in tokens_temp if (word not in stop_words_temp and word not in punctuation_list_temp and word.isnumeric()==False)]\nRESULT=[\'name\', \'ayush\', \'fir.st\', \'second\', \'.ninth\']\n\n#EXAMPLE TO SHOW WORKING OF ASCII REMOVAL IN PYTHON\nstring_with_nonASCII = "àa string withé fuünny charactersß."\nencoded_string = string_with_nonASCII.encode("ascii", "ignore")\ndecode_string = encoded_string.decode()\nprint(decode_string)\nRESULT="a string with funny characters."\n\n'

In [205]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import time
import string
import os
stop_words = stopwords.words('english')

In [41]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import time
import string
import os
documents={}
start=time.time()
stop_words = stopwords.words('english')     #stopwords in english langauge
my_dict={}   # dictionary to store tokens and doc_id pairs
#reading all the files in folder english-corpora
curr_folder=os.path.join(os.getcwd(),"english-corpora")
for root,folders,files in os.walk(curr_folder):
    
    for file in files:
        path=os.path.join(root,file)
        file_content= open(path, encoding='utf8').read()
        
        #fetch document id
        doc_id=re.sub('.txt','',file)
        

        #tokenization
        tokens = nltk.word_tokenize(file_content) 
        #remove punctuations
        tokens=[word.translate(str.maketrans('','',string.punctuation)) for word in tokens]
        #remove digits
        tokens=[word.translate(str.maketrans('','','1234567890')) for word in tokens]
        
        tokens=[word for word in tokens if word!='']

        tokens =[word.lower() for word in tokens if word not in stop_words]
        

        #removal of non-ascii characters
        for i in range(0,len(tokens)):
            temp=tokens[i].encode('ascii','ignore')
            tokens[i]=temp.decode()

        #stemming
        ps=PorterStemmer()
        tokens=[ps.stem(word) for word in tokens]
              
        for word in tokens:
            if word in my_dict:
                my_dict[word].add(doc_id)
            else:
                my_dict[word]=set()
                my_dict[word].add(doc_id)
            
        documents[doc_id]=tokens
        
        
        
end=time.time()
print(end-start)

2226.236476659775


In [1]:
#saving dictionary to pickle file for fast loading 
import pickle

#posting list dictionary : 
#file=open('tokens-dict.pkl','wb')
#pickle.dump(my_dict,file)
#file.close()

file=open('tokens-dict.pkl','rb')
posting_list=pickle.load(file)
file.close()


#documents dictionary:
#file=open('doc-dict.pkl','wb')
#pickle.dump(documents,file)
#file.close()

file=open('doc-dict.pkl','rb')
documents_list=pickle.load(file)
file.close()


In [206]:
#function to refine the query
def preprocess(query):
    query_tokens=nltk.word_tokenize(query)
    query_tokens=[word.lower() for word in query_tokens] 
    query_tokens=[word.translate(str.maketrans('','',string.punctuation)) for word in query_tokens]
    query_tokens=[word.translate(str.maketrans('','','1234567890')) for word in query_tokens]
        
    query_tokens=[word for word in query_tokens if word!='']

    query_tokens =[word.lower() for word in query_tokens if word not in stop_words]   #lowercase
        

    #removal of non-ascii characters
    for i in range(0,len(query_tokens)):
        temp=query_tokens[i].encode('ascii','ignore')
        query_tokens[i]=temp.decode()

    #stemming
    ps=PorterStemmer()
    query_tokens=[ps.stem(word) for word in query_tokens]
    
    query_tokens=[word for word in query_tokens if word in posting_list]
    return query_tokens

In [253]:
#creating term requency dictionary
tf_dict={}

for key in documents_list:
    for word in documents_list[key]:
        if word not in tf_dict:
            tf_dict[word]={}
        else:
            if key not in tf_dict[word]:
                tf_dict[word][key]=1
            else:
                tf_dict[word][key]=tf_dict[word][key]+1

for word in tf_dict:
    for doc_id in tf_dict[word]:
        tf_dict[word][doc_id]=tf_dict[word][doc_id]/len(documents_list[doc_id])

# BOOLEAN RETRIEVAL 

In [207]:
#BOOLEAN RETRIEVAL MODEL
def boolean_retrieval_func(query):
    query_tokens=preprocess(query)
    
    temp_set1=posting_list[query_tokens[0]]     #giving reference of a set to temp_set1
    for i in range(1,len(query_tokens)):
        temp_set2=posting_list[query_tokens[i]]
        temp_set1=temp_set1 & temp_set2         #performing intersection of sets(very fast operation)
    
    top_10=[]
    for doc_id in temp_set1:
        score=0
        for word in query_tokens:
            if doc_id not in tf_dict[word]:
                continue
            score+=tf_dict[word][doc_id]
            
        top_10.append((score,doc_id))
        
    top_10.sort(reverse=True)
    res=[]
    for i in range(min(10,len(top_10))):
        res.append(top_10[i][1])
    return res

In [7]:
import math
#creating idf dictionary
idf_dict={}
n_docs=8635

for word in posting_list:
    n_docs_curr_word=len(posting_list[word])
    idf_dict[word]=math.log(n_docs/n_docs_curr_word)
    

    

In [104]:
vocab_dict={}    #dictionary to store words and their correspondihng index(to be used in calculation of cosine similarity)
i=0
for word in posting_list:
    vocab_dict[word]=i
    i=i+1

    


In [116]:
#average length of documents which is required in BM25
L=0
for doc in documents_list:
    L=L+len(documents_list[doc])
    #print(len(doc))
    
L=L/len(documents_list)

len(documents_list)
print(L)

2243.5438332368267


In [93]:
%%time
#storing unique words present in each document
doc_unique = {}

for doc in documents_list : 
    doc_unique[doc] = set()
    for word in documents_list[doc] :
        doc_unique[doc].add(word)

CPU times: total: 9.47 s
Wall time: 10 s


# TF-IDF

In [134]:
import numpy as np
vocab_len=len(posting_list)  #no of unique words in all the documents
A=np.zeros((vocab_len))

#function to compute tf-idf value of each document corresponding to given query
def tfidf(query):
    cosine_similarity=[]
    query_tokens= preprocess(query)
    B=np.zeros((vocab_len))
    tf_query={}
    for word in query_tokens:
        tf_query[word]=0
    for word in query_tokens:
        tf_query[word]+=1
    for word in query_tokens:
        tf_query[word]/=len(query_tokens)
        
    for word in query_tokens:
        word_index=vocab_dict[word]
        B[word_index]=tf_query[word]*idf_dict[word]
        
    normB=np.linalg.norm(B)
    documents_imp = set()
    for word in query_tokens :
        for doc_id in posting_list[word]:
            documents_imp.add(doc_id)
    
    
    for doc_id in documents_imp:
        temp = np.zeros((vocab_len))
        for word in doc_unique[doc_id]:
            if doc_id not in tf_dict[word]:
                continue
            tf_value=tf_dict[word][doc_id]
            word_index=vocab_dict[word]
            temp[word_index]=idf_dict[word]*tf_value
        normTemp=np.linalg.norm(temp)
        cosine_score=np.dot(temp,B)/(normTemp*normB)
        cosine_similarity.append((cosine_score,doc_id))
        
    cosine_similarity.sort(reverse=True)
    ans=[]
    n=len(cosine_similarity)
    for i in range(min(10,len(cosine_similarity))):
        ans.append(cosine_similarity[i][1])

    return ans

In [145]:
%%time
ans=tfidf("Google made a wide deployment of standards")
print(ans)

['C00030', 'S00003', 'C00743', 'C00015', 'S00407', 'C00296', 'C00551', 'C00021', 'C00762', 'C00678']
CPU times: total: 42.8 s
Wall time: 23.2 s


# BM25

In [132]:
#function to calculate bm25 score of each document corresponding to given query
def BM25(query,k=2,b=0.75):
    query_tokens=preprocess(query)
    result=[]
    N=len(documents_list)
    documents_imp = set()
    for word in query_tokens :
        for doc_id in posting_list[word]:
            documents_imp.add(doc_id)    #document_imp denotes the documents in which current token of query is present
    for doc_id in documents_imp:
        doc_score=0
        for word in query_tokens:
            if doc_id not in tf_dict[word]:
                continue
            x_idf=(N - len(posting_list[word]) + 0.5)/(len(posting_list[word])+0.5)
            x_idf=math.log(x_idf)
            
            second_term=(tf_dict[word][doc_id]*len(documents_list[doc_id]) * (k+1) )/(tf_dict[word][doc_id]*len(documents_list[doc_id]) + k*(1-b+b*len(documents_list[doc_id])/L))
            prod=x_idf*second_term
            doc_score+=prod
            
        result.append((doc_score,doc_id))
        
    result.sort(reverse=True)
    ans=[]
    for i in range(min(10,len(result))):
        ans.append(result[i][1])
        
    return ans

In [147]:
%%time
#CHECKING SOME QUERIES FOR RESULTS
ans=BM25("Internet of things describes physical objects")
print(ans)

['C00017', 'C00519', 'C00345', 'M00003', 'S00180', 'C00387', 'C00795', 'D00828', 'S00063', 'S00073']
CPU times: total: 46.9 ms
Wall time: 59.8 ms


In [214]:
#CHECKING SOME RESULTS
boolean_retrieval_func("replacing the absent hormones")

['D00985',
 'D00488',
 'D00101',
 'D00549',
 'D00681',
 'D00171',
 'D00428',
 'D00886',
 'D00258',
 'D00141']

In [252]:
''' TO GENERATE QRELS
file_read=open('queries.txt')
a=file_read.readlines()
file_write=open('qrels.txt','a')
for line in a:
    qid=re.sub('\t.*','',line)
    qid=re.sub('\n','',qid)
    stringh=re.sub('Q[0-9][0-9]\t','',line)
    docs_bm=BM25(stringh)
    for doc_id in docs_bm:
        file_write.write(qid+', 1, '+doc_id+', 1\n')
    
file_read.close()
file_write.close()
'''